In [1]:
%run init_notebook.py

In [2]:
from src.utils import load_pd_df, get_dt_index

In [4]:
pi = get_dt_index(load_pd_df('de_inf_prepro.csv'))
pi_e = load_pd_df('infl_expec.feather')
bei = load_pd_df('bei_data.feather')

In [5]:
pi['M'] = pi.index.to_period('M').start_time
pi_e['M'] = pi_e.index.to_period('M').start_time
pi_e.index.name = 'date'
bei['W'] = bei.index.to_period('W').start_time
bei.drop('pi_de_surprise_Y', axis=1, inplace=True)
bei = bei.reset_index().set_index('W')

In [6]:
pi = pi.reset_index().set_index('M')[['pi_de_Y', 'pi_de_surprise_Y', 'date']]
pi_e = pi_e.reset_index().set_index('M')[['pi_e_estimate_mean']]

In [7]:
pi = pi.set_index(pi.date.dt.to_period('W'))
pi.index = pi.index.start_time

In [8]:
pi = pi.join(pi_e)
pi['W'] = pi.date.dt.to_period('W')

In [9]:
pi = pi.set_index('W')
pi.index = pi.index.start_time

In [10]:
df = bei[['date', 'bei']].join(pi.drop('date', axis=1)).copy()

In [12]:
df.describe().drop('date', axis=1).to_csv(os.path.join(GRAPHS_DIR, 'desc.csv'))
df.drop('date', axis=1).cov().to_csv(os.path.join(GRAPHS_DIR, 'cov.csv'))

In [13]:
df.drop('date', axis=1).cov()

,bei,pi_de_Y,pi_de_surprise_Y,pi_e_estimate_mean
bei,0.000029,0.000158,0.000163,0.000085
pi_de_Y,0.000158,0.001128,0.001166,0.000605
pi_de_surprise_Y,0.000163,0.001166,0.001210,0.000601
pi_e_estimate_mean,0.000085,0.000605,0.000601,0.000486


In [180]:
with Capturing() as out:
    print(df.describe().drop('date', axis=1))
write_to_txt("\n".join(out), 'desc.txt')

with Capturing() as out:
    print(df.drop('date', axis=1).cov())
write_to_txt("\n".join(out), 'cov.txt')